# Mini Project 2: IMDB Sentiment Analysis

February 22, 2019

Akshal Aniche, Jacob Sanz-Robinson, Raphael Hotter

COMP 551

## Imports

In [27]:
import os
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

## Data processer preparation

In [3]:
count_vec = CountVectorizer()

## Load the data

In [4]:
pos_train_files = os.listdir('data/train/pos/')
neg_train_files = os.listdir('data/train/neg/')
test_files = os.listdir('data/test/')

# Remove .DS_Store files
while '.DS_Store' in pos_train_files:
  pos_train_files.remove('.DS_Store')
while '.DS_Store' in neg_train_files:
  neg_train_files.remove('.DS_Store')
while '.DS_Store' in test_files:
  test_files.remove('.DS_Store')

In [ ]:
# Reads data into 2 arrays
train_text, test_raw = [], []

for file in pos_train_files:
  with open('data/train/pos/{}'.format(file), 'r') as f:
    train_text.append(f.read())

for file in neg_train_files:
  with open('data/train/neg/{}'.format(file), 'r') as f:
    train_text.append(f.read())

for file in test_files:
  with open('data/test/{}'.format(file), 'r') as f:
    test_raw.append(f.read())

In [ ]:
# Count the word occurrences in the training and test datasets (bag of words)
count_vec.fit(train_text)
train_bow = count_vec.transform(train_text)
test_bow = count_vec.transform(test_raw)

In [37]:
# Training set Y vector
pos_goal = np.ones((12500, 1))
neg_goal = np.zeros((12500, 1))
Y_train = np.append(pos_goal, neg_goal, axis = 0)

## Text processing